#### Importar libs

In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo
import requests
import pandas as pd
from io import BytesIO

import os
import sys

sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

#### Atualizar o ROA e os volumes detalhados

In [2]:
conexao = Connect.connect_techdb()
hist = Connect.import_table(conexao, "previa_receita_nova")
conexao.close()

In [3]:
brasilia_tz = ZoneInfo('America/Sao_Paulo')

hist = hist[hist['Data'] != datetime.now(brasilia_tz).strftime("%Y-%m")]

In [ ]:
def load_previas(advisor_name, link: str):
    response = \
        requests.get("{}".format(link),
                      params={"downloadformat": "excel"})
    df = pd.read_excel(BytesIO(response.content), sheet_name='Meta')
    df.rename(columns={df.columns[2]:"REALIZADO - VOLUME",
                    df.columns[5]:"REALIZADO - ROA"}, inplace=True)
    df.drop(["%", "%.1"], axis=1, inplace=True)
    df = df[df['Categoria - Acompanhamento Next'] != "TOTAL"]
    df['Assessor'] = advisor_name
    return df



In [5]:
rodrigo = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQBVuGicHybdRrC4d1MtFO8vAbY4Kw4m4_8gNo8EKu3BN4I?download=1", advisor_name="RODRIGO DE MELLO D’ELIA")
caic = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQByqpoVZXN3TYdcUlFmYqE9Af8AsDkk6umaNM26wLfQlo4?download=1", advisor_name="CAIC ZEM GOMES")
fernando = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQAFP5cFxU9QRL_OWClG-BJdAT3Wvk18_VoypIF3CxIpQYY?download=1", advisor_name="FERNANDO DOMINGUES DA SILVA")
saadallah = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQDjXAOoHHxgQYo_aeQQdXz4AXzuH0TotDouMPG_NNH4H-4?download=1", advisor_name="SAADALLAH JOSE ASSAD")
paulo = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQA6mQn9Z8lwTL7HfLdw1UzSAbCoa8qOg03wNZvbJyDoIaw?download=1", advisor_name="PAULO ROBERTO FARIA SILVA")
# guerrero = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQAtEitVDNlnT4zWgSIonhfzAQlTKUlOX3xzRRYfgcoHrZE?download=1", advisor_name="GABRIEL GUERRERO TORRES FONSECA")
renan_bento = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQCr2UivmbvdQ4EIlpS8fcTfAVmuuqLXeLhqroDonRcYXDQ?download=1", advisor_name = "RENAN BENTO DA SILVA")
rosana = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQDmCPZulmfjTYtC94MGxHh1ARzYBjyg9gWNL9v2U--M0CQ?download=1", advisor_name = "ROSANA APARECIDA PAVANI DA SILVA")
rafael_pasold = load_previas(link="https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQBVpoNJYZ8NSY9rnRotpPSPAXlir-PVsE7SItom19ZhijI?download=1", advisor_name = "RAFAEL PASOLD MEDEIROS")


In [6]:
previa_receita = pd.concat([paulo, rodrigo, fernando, rafael_pasold, caic, rosana,  saadallah, renan_bento], ignore_index=True)

previa_receita['Data'] = pd.to_datetime(datetime.now(brasilia_tz).strftime("%Y-%m"))
previa_receita['Hora Atualizado'] = pd.to_datetime(datetime.now(brasilia_tz).strftime("%Y-%m-%d %H:%M:%S"))

In [7]:
previa_receita = pd.concat([hist, previa_receita], axis=0)

In [8]:
previa_receita.loc[previa_receita['META - VOLUME'] == '-', 'META - VOLUME'] = 0
previa_receita.loc[previa_receita['META - ROA'] == '-', 'META - ROA'] = 0

In [9]:
conexao = Connect.connect_techdb()
previa_receita.to_sql('previa_receita_nova', 
                    conexao, 
                    if_exists='replace', 
                    index=False,
                    schema="principal")
conexao.close()

#### Atualizar receita previa por assessor

In [10]:
previa_receita_assessor = previa_receita.loc[:, ["Assessor", "Data", "META - ROA", "REALIZADO - ROA"]].groupby(['Assessor', 'Data']).sum().reset_index()

In [11]:
conexao = Connect.connect_techdb()
historico_previa_receita = Connect.import_table(conexao, "previa_receita_assessor_historico")
conexao.close()

In [12]:
# %%
# 1. Normaliza as datas para garantir que a comparação funcione
historico_previa_receita['Data'] = (
    pd.to_datetime(historico_previa_receita['Data'])
    .dt.to_period('M')
    .dt.to_timestamp()
)

previa_receita_assessor['Data'] = (
    pd.to_datetime(previa_receita_assessor['Data'])
    .dt.to_period('M')
    .dt.to_timestamp()
)

# --- DEBUG 1: O que veio do banco? ---
print("--- [DEBUG] Antes do Filtro ---")
print(f"Linhas no histórico: {len(historico_previa_receita)}")
# Mostra os meses únicos ordenados para você ver fácil se Dezembro tá lá
print("Meses presentes no histórico:", sorted(historico_previa_receita['Data'].unique())) 
print("-" * 30)


# 2. Define mês atual 
mes_atual = (
    pd.Timestamp.now(tz=brasilia_tz)
    .to_period('M') 
    .to_timestamp()
)
# Obs: Se pd.Timestamp.now() retornar um timestamp direto, usa-se .to_period('M').to_timestamp() sem o .dt
# Ajuste seguro para o seu snippet:
mes_atual = pd.Timestamp.now(tz=brasilia_tz).to_period('M').to_timestamp()

print(f"--- [DEBUG] Mês de Corte (Atual): {mes_atual} ---")


# 3. Remove APENAS o mês atual do histórico
historico_previa_receita = historico_previa_receita[
    historico_previa_receita['Data'] != mes_atual
]

# --- DEBUG 2: O filtro funcionou? ---
# Aqui garantimos que o mês atual sumiu, mas o anterior (Dezembro) ficou
print("--- [DEBUG] Pós Filtro (Histórico Limpo) ---")
print("Meses que restaram no histórico:", sorted(historico_previa_receita['Data'].unique()))


# 4. Seleciona apenas o mês atual calculado da nova tabela
previa_receita_mes_atual = previa_receita_assessor[
    previa_receita_assessor['Data'] == mes_atual
]
print(f"--- [DEBUG] Novas linhas de {mes_atual}: {len(previa_receita_mes_atual)} ---")


# 5. Reconstrói histórico
previa_receita_assessor_historico = pd.concat(
    [historico_previa_receita, previa_receita_mes_atual],
    axis=0
)

print(f"--- [DEBUG] Final: {len(previa_receita_assessor_historico)} linhas prontas para subir ---")

--- [DEBUG] Antes do Filtro ---
Linhas no histórico: 109
Meses presentes no histórico: [Timestamp('2025-03-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2025-06-01 00:00:00'), Timestamp('2025-07-01 00:00:00'), Timestamp('2025-08-01 00:00:00'), Timestamp('2025-09-01 00:00:00'), Timestamp('2025-10-01 00:00:00'), Timestamp('2025-11-01 00:00:00'), Timestamp('2025-12-01 00:00:00'), Timestamp('2026-01-01 00:00:00')]
------------------------------
--- [DEBUG] Mês de Corte (Atual): 2026-01-01 00:00:00 ---
--- [DEBUG] Pós Filtro (Histórico Limpo) ---
Meses que restaram no histórico: [Timestamp('2025-03-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2025-06-01 00:00:00'), Timestamp('2025-07-01 00:00:00'), Timestamp('2025-08-01 00:00:00'), Timestamp('2025-09-01 00:00:00'), Timestamp('2025-10-01 00:00:00'), Timestamp('2025-11-01 00:00:00'), Timestamp('2025-12-01 00:00:00')]
--- [DEBUG] Novas linhas de 20

C:\Users\joaob\AppData\Local\Temp\ipykernel_13880\1154988934.py:26: UserWarning: Converting to Period representation will drop timezone information.
  .to_period('M')
C:\Users\joaob\AppData\Local\Temp\ipykernel_13880\1154988934.py:31: UserWarning: Converting to Period representation will drop timezone information.
  mes_atual = pd.Timestamp.now(tz=brasilia_tz).to_period('M').to_timestamp()


In [13]:
conexao = Connect.connect_techdb()
previa_receita_assessor_historico.to_sql(
    'previa_receita_assessor_historico',
    conexao,
    if_exists='replace',
    index=False,
    schema="principal"
)
conexao.close()